# 2.1 Fundamentos de Window Functions\n\n- Objetivo: usar OVER() para cálculos fila-a-fila y rankings.\n- Prerrequisitos: ventas pobladas en `fact_ventas`.\n- Ejercicios: RANK(), DENSE_RANK(), LAG/LEAD.\n- Reto: variación porcentual robusta evitando división por cero.\n- Errores comunes: olvidar PARTITION BY, ORDER BY incorrecto en LAG/LEAD.

## ¿Por qué usar Window Functions?

Las funciones de ventana (window functions) permiten realizar cálculos sobre conjuntos de filas **relacionadas** con la fila actual, SIN colapsar las filas como hace GROUP BY.

**Problema con GROUP BY:**
```sql
SELECT producto_id, SUM(cantidad) AS total
FROM fact_ventas
GROUP BY producto_id;
```
**Resultado:** Una fila por producto (colapsa todas las ventas de ese producto en una sola línea).

**Solución con Window Functions:**
```sql
SELECT producto_id, fecha, cantidad,
       SUM(cantidad) OVER (PARTITION BY producto_id) AS total_producto
FROM fact_ventas;
```
**Resultado:** Todas las filas originales + columna calculada con el total del producto.

**Para qué:**
- Rankings sin eliminar filas (top 10 productos manteniendo todas sus ventas)
- Acumulados running totals (ventas acumuladas día a día)
- Comparaciones período anterior (LAG: ventas de hoy vs ayer)
- Promedios móviles (últimos 7 días)

**Cómo:**
- **OVER():** Define la "ventana" de filas para el cálculo
- **PARTITION BY:** Reinicia cálculo por cada grupo (como GROUP BY pero sin colapsar)
- **ORDER BY:** Define orden para cálculos secuenciales (LAG, acumulados)
- **ROWS/RANGE:** Especifica rango de filas para funciones agregadas

## Tipos de Window Functions

### 1. Funciones de Ranking

#### RANK()
Asigna posición con saltos cuando hay empates.
```sql
SELECT producto_id, ventas,
       RANK() OVER (ORDER BY ventas DESC) AS ranking
FROM productos_ventas;
```
**Ejemplo:** 1, 2, 2, 4 (dos productos empatados en 2do lugar, siguiente es 4to)

#### DENSE_RANK()
Asigna posición sin saltos.
```sql
DENSE_RANK() OVER (ORDER BY ventas DESC) AS ranking
```
**Ejemplo:** 1, 2, 2, 3 (siguiente al empate es 3ro, no 4to)

#### ROW_NUMBER()
Asigna número secuencial único (desempata arbitrariamente).
```sql
ROW_NUMBER() OVER (ORDER BY ventas DESC) AS fila
```
**Ejemplo:** 1, 2, 3, 4 (no hay empates, desempata por orden interno)

**¿Cuál usar?**
- **ROW_NUMBER():** Cuando necesitas IDs únicos o paginar resultados
- **RANK():** Cuando quieres mostrar saltos en ranking (como en deportes)
- **DENSE_RANK():** Cuando quieres posiciones consecutivas (top 10 productos puede incluir más de 10 si hay empates)

### 2. Funciones de Acceso (LAG/LEAD)

#### LAG() - Valor de fila anterior
```sql
SELECT fecha, ventas,
       LAG(ventas, 1, 0) OVER (ORDER BY fecha) AS ventas_ayer
FROM ventas_diarias;
```
**Parámetros:**
- `LAG(columna, offset, default)`
- `offset`: cuántas filas atrás (1 = fila anterior, 2 = dos filas atrás)
- `default`: valor si no hay fila anterior (primera fila)

#### LEAD() - Valor de fila siguiente
```sql
SELECT fecha, ventas,
       LEAD(ventas) OVER (ORDER BY fecha) AS ventas_manana
FROM ventas_diarias;
```

**Caso de uso: Variación porcentual**
```sql
SELECT fecha, ventas,
       LAG(ventas) OVER (ORDER BY fecha) AS ventas_previas,
       CASE 
         WHEN LAG(ventas) OVER (ORDER BY fecha) > 0 
         THEN (ventas - LAG(ventas) OVER (ORDER BY fecha)) * 100.0 / LAG(ventas) OVER (ORDER BY fecha)
         ELSE NULL
       END AS variacion_pct
FROM ventas_diarias;
```

### 3. Funciones Agregadas con OVER()

#### Running Total (Acumulado)
```sql
SELECT fecha, ventas,
       SUM(ventas) OVER (ORDER BY fecha ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS acumulado
FROM ventas_diarias;
```
**ROWS BETWEEN:**
- `UNBOUNDED PRECEDING`: desde el inicio
- `CURRENT ROW`: hasta la fila actual
- `UNBOUNDED FOLLOWING`: hasta el final

#### Promedio Móvil (Moving Average)
```sql
SELECT fecha, ventas,
       AVG(ventas) OVER (ORDER BY fecha ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS promedio_7dias
FROM ventas_diarias;
```
**Interpretación:** Promedio de últimos 7 días (fila actual + 6 anteriores)

## PARTITION BY - Reiniciar por Grupo

Sin PARTITION BY:
```sql
SELECT producto_id, fecha, cantidad,
       SUM(cantidad) OVER (ORDER BY fecha) AS acumulado_global
FROM fact_ventas;
```
**Resultado:** Acumulado de TODOS los productos juntos

Con PARTITION BY:
```sql
SELECT producto_id, fecha, cantidad,
       SUM(cantidad) OVER (PARTITION BY producto_id ORDER BY fecha) AS acumulado_producto
FROM fact_ventas;
```
**Resultado:** Acumulado reinicia para cada producto (acumulado por producto)

**Analogía:** PARTITION BY es como hacer múltiples cálculos independientes, uno por cada valor de la columna especificada.

## Errores Comunes

1. **Olvidar PARTITION BY cuando se necesita:**
   ```sql
   -- INCORRECTO: ranking global cuando querías ranking por categoría
   RANK() OVER (ORDER BY ventas DESC)
   
   -- CORRECTO: ranking dentro de cada categoría
   RANK() OVER (PARTITION BY categoria ORDER BY ventas DESC)
   ```

2. **ORDER BY incorrecto en LAG/LEAD:**
   Si comparas "ventas de hoy vs ayer", ORDER BY debe ser por `fecha`, no por `ventas`.

3. **Llamar window function múltiples veces:**
   ```sql
   -- INEFICIENTE:
   LAG(ventas) OVER (...), 
   (ventas - LAG(ventas) OVER (...)) / LAG(ventas) OVER (...)
   
   -- MEJOR: calcular LAG una vez en subconsulta o CTE
   ```

4. **División por cero en variaciones:**
   Siempre usar CASE para verificar denominador > 0 antes de dividir.

In [ ]:
-- Ranking de productos por unidades vendidas (desc)
SELECT producto_id, SUM(cantidad) AS unidades,
       RANK() OVER (ORDER BY SUM(cantidad) DESC) AS rk
FROM dbo.fact_ventas
GROUP BY producto_id;

In [ ]:
-- Ventas por día y acumulado
SELECT fecha, SUM(cantidad) AS unidades_dia,
       SUM(SUM(cantidad)) OVER (ORDER BY fecha ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS unidades_acumuladas
FROM dbo.fact_ventas
GROUP BY fecha
ORDER BY fecha;

🟢 Ejercicio: Agregar DENSE_RANK por margen total de producto.

In [ ]:
SELECT p.producto_id, p.nombre,
       SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen_total,
       DENSE_RANK() OVER (ORDER BY SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) DESC) AS drk
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY p.producto_id, p.nombre;

🟠 Ejercicio: Usar LAG para comparar unidades de cada producto vs día anterior.

In [ ]:
SELECT producto_id, fecha, SUM(cantidad) AS unidades_dia,
       LAG(SUM(cantidad)) OVER (PARTITION BY producto_id ORDER BY fecha) AS unidades_previas
FROM dbo.fact_ventas
GROUP BY producto_id, fecha
ORDER BY producto_id, fecha;

🔴 Reto: Calcular la variación porcentual diaria acumulada de margen usando una combinación de SUM(), LAG y expresión CASE para evitar división por cero.

Errores comunes: olvidar PARTITION BY; mezclar filtros fuera del window sub-optimizing; no ordenar correctamente para LAG/LEAD.